<a href="https://colab.research.google.com/github/yuuOkada/Sphere_CNN/blob/master/Video_show_predict_result.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Time elapsed since Colab Instance was started: Shutdown in 0.5day(43200 second)

In [0]:
!cat /proc/uptime | awk '{print $1 /60 /60 /24 "days (" $1 "sec)"}'

0.00399514days (345.18sec)


Connect to Google Drive File Stream


In [1]:
from google.colab import drive
drive.mount('./gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at ./gdrive


Sphere Predict (11:9) , *shape(90,110,3) , 8Class*

In [0]:
import os
import cv2
import time
import numpy as np
import tensorflow as tf
from PIL import Image
import matplotlib.pyplot as plt
from tensorflow import keras
from keras import models
from keras.models import model_from_json
from keras.preprocessing import image

root_dir = "./gdrive/My Drive/Google Colab/CO191219_proj"

#Load saved-model
model = model_from_json(open(root_dir+'/Conv2D3-Dence3-Adadelta_v2/img_predict_Conv2D3-Dence3-Adadelta.json').read())
#Load saved-weight
model.load_weights(root_dir+'/Conv2D3-Dence3-Adadelta_v2/img_predict_Conv2D3-Dence3-Adadelta.hdf5')
#Define categories
categories = ["ceiling", "cloth", "floor", "floor-object","hole", "human", "object", "wall"]

#Define constant
ESC_KEY = 27     # Esc Key
INTERVAL= 1     # Interval
FRAME_RATE = 5  #Video fps

#Load original video file
mov_org = cv2.VideoCapture(root_dir + "/demovideo/demovideo.m4v")
ret, frame = mov_org.read()

#Define Output-video properties
font = cv2.FONT_HERSHEY_PLAIN #Font for output-video
font_size = 1.8 #Font size for output-video
font_boldness = 2 #Font Boldness for output-video
width = int(mov_org.get(cv2.CAP_PROP_FRAME_WIDTH)) #Get original movie width
height = int(mov_org.get(cv2.CAP_PROP_FRAME_HEIGHT)) #Get original movie height
fps = mov_org.get(cv2.CAP_PROP_FPS) #Get original movie width
fourcc = cv2.VideoWriter_fourcc(*'XVID') #Define Videowriter in opencv for output
#mov_out = cv2.VideoWriter(root_dir + "/demovideo/demovideo_out_v1.mp4", fourcc, fps, (width,height)) #Define output-video codec
mov_out = cv2.VideoWriter(root_dir + "/demovideo/demovideo_out_v1.m4v", fourcc, fps, (width,height)) #Define output-video codec

#Loop while original video is remaining
while ret == True:
    #Shift int-type to float-type
    flt32_frame = frame.astype(np.float32)
    
    #Define numpy-array for predict-result
    predict_result = np.empty(0)

    #Process entire image by CNN
    for vartical_index in range(0, 972, 108):
        for horizontal_index in range(15, 1770, 135):
          
          #print(str(horizontal_index) + "," + str(vartical_index))

          if ((vartical_index % 216 == 0) and ((horizontal_index-15) % 270 == 0)) or vartical_index >= 540:

              fr = frame[vartical_index:vartical_index+215, horizontal_index:horizontal_index+269]
              cv2.imwrite(root_dir  + "/demovideo/temp_frame.jpg", fr)

              img = image.load_img(root_dir  + "/demovideo/temp_frame.jpg", target_size=(90, 110, 3))
              x = image.img_to_array(img)
              x = np.expand_dims(x, axis=0)

              features = model.predict(x)
              #print(features)
              max_features = np.amax(features)

              if max_features == 0:
                  predict_result = np.append(predict_result, 0)
                  cv2.circle(frame, (horizontal_index+135, vartical_index+108), 5, (244,121,128), thickness=-1)
                  cv2.putText(frame, "error", (horizontal_index+135+5, vartical_index+108+5), font, font_size,(244,121,128), font_boldness)
              elif features[0,0] == max_features:
                  predict_result = np.append(predict_result, 1)
                  cv2.circle(frame, (horizontal_index+135, vartical_index+108), 5,(164,142,199), thickness=-1)
                  cv2.putText(frame, categories[0], (horizontal_index+135+5, vartical_index+108+5), font, font_size,(164,142,199), font_boldness)
              elif features[0,1] == max_features:
                  predict_result = np.append(predict_result, 2)
                  cv2.circle(frame, (horizontal_index+135, vartical_index+108), 5,(164,142,199), thickness=-1)
                  cv2.putText(frame, categories[1], (horizontal_index+135+5, vartical_index+108+5), font, font_size,(164,142,199), font_boldness)
              elif features[0,2] == max_features:
                  predict_result = np.append(predict_result, 3)
                  cv2.circle(frame, (horizontal_index+135, vartical_index+108), 5, (113,208,255), thickness=-1)
                  cv2.putText(frame, categories[2], (horizontal_index+135+5, vartical_index+108+5), font, font_size,(113,208,255), font_boldness)
              elif features[0,3] == max_features:
                  predict_result = np.append(predict_result, 4)
                  cv2.circle(frame, (horizontal_index+135, vartical_index+108), 5, (244,121,128), thickness=-1)
                  cv2.putText(frame, categories[3], (horizontal_index+135+5, vartical_index+108+5), font, font_size,(244,121,128), font_boldness)
              elif features[0,4] == max_features:
                  predict_result = np.append(predict_result, 5)
                  cv2.circle(frame, (horizontal_index+135, vartical_index+108), 5, (244,121,128), thickness=-1)
                  cv2.putText(frame, categories[4], (horizontal_index+135+5, vartical_index+108+5), font, font_size,(244,121,128), font_boldness)
              elif features[0,5] == max_features:
                  predict_result = np.append(predict_result, 6)
                  cv2.circle(frame, (horizontal_index+135, vartical_index+108), 5, (244,121,128), thickness=-1)
                  cv2.putText(frame, categories[5], (horizontal_index+135+5, vartical_index+108+5), font, font_size,(244,121,128), font_boldness)
              elif features[0,6] == max_features:
                  predict_result = np.append(predict_result, 7)
                  cv2.circle(frame, (horizontal_index+135, vartical_index+108), 5,(244,121,128), thickness=-1)
                  cv2.putText(frame, categories[6], (horizontal_index+135+3, vartical_index+108+5), font, font_size,(244,121,128), font_boldness)
              elif features[0,7] == max_features:
                  predict_result = np.append(predict_result, 8)
                  cv2.circle(frame, (horizontal_index+135, vartical_index+108), 5, (244,121,128), thickness=-1)
                  cv2.putText(frame, categories[7], (horizontal_index+135+5, vartical_index+108+5), font, font_size,(244,121,128), font_boldness)
              else:
                  predict_result = np.append(predict_result, 0)
                  cv2.circle(frame, (horizontal_index+135, vartical_index+108), 5, (244,121,128), thickness=-1)
                  cv2.putText(frame, "error", (horizontal_index+135+5, vartical_index+108+5), font, font_size,(244,121,128), font_boldness)

          else:
              predict_result = np.append(predict_result, 0)
              #print("out")

    predict_result = predict_result.reshape(9,13)
    #print(predict_result)

    #Write for movie
    mov_out.write(frame)

    #if Esc Key is pushed, Escape loop
    key = cv2.waitKey(INTERVAL)
    if key == ESC_KEY:
        break

    #Load next video-frame
    ret, frame = mov_org.read()

#End
cv2.destroyAllWindows()
mov_org.release()
mov_out.release()

Sphere Predict (11:9), *shape(60,70,3) , 7-false8-Class*

In [2]:
import os
import cv2
import time
import numpy as np
import tensorflow as tf
from PIL import Image
import matplotlib.pyplot as plt
from tensorflow import keras
from keras import models
from keras.models import model_from_json
from keras.preprocessing import image

root_dir = "./gdrive/My Drive/Google Colab/CO191219_proj"

#Load saved-model
model = model_from_json(open(root_dir+'/Conv2D3-Dence3-Adadelta_v4/img_predict_Conv2D3-Dence3-Adadelta.json').read())
#Load saved-weight
model.load_weights(root_dir+'/Conv2D3-Dence3-Adadelta_v4/img_predict_Conv2D3-Dence3-Adadelta.hdf5')
#Define categories
categories = ["floor1", "floor2", "floor-object", "object", "wall", "hole", "human","ceiling"]

#Define constant
ESC_KEY = 27     # Esc Key
INTERVAL= 1     # Interval
FRAME_RATE = 5  #Video fps

#Load original video file
mov_org = cv2.VideoCapture(root_dir + "/demovideo/demovideo_short.m4v")
ret, frame = mov_org.read()

#Define Output-video properties
font = cv2.FONT_HERSHEY_PLAIN #Font for output-video
font_size = 1.8 #Font size for output-video
font_boldness = 2 #Font Boldness for output-video
width = int(mov_org.get(cv2.CAP_PROP_FRAME_WIDTH)) #Get original movie width
height = int(mov_org.get(cv2.CAP_PROP_FRAME_HEIGHT)) #Get original movie height
fps = mov_org.get(cv2.CAP_PROP_FPS) #Get original movie width
fourcc = cv2.VideoWriter_fourcc(*'XVID') #Define Videowriter in opencv for output
#mov_out = cv2.VideoWriter(root_dir + "/demovideo/demovideo_out_v1.mp4", fourcc, fps, (width,height)) #Define output-video codec
mov_out = cv2.VideoWriter(root_dir + "/demovideo/demovideo_out_v3.m4v", fourcc, fps, (width,height)) #Define output-video codec

#Loop while original video is remaining
while ret == True:
    #Shift int-type to float-type
    flt32_frame = frame.astype(np.float32)
    
    #Define numpy-array for predict-result
    predict_result = np.empty(0)

    #Process entire image by CNN
    for vartical_index in range(0, 972, 108):
        for horizontal_index in range(15, 1770, 135):
          
          #print(str(horizontal_index) + "," + str(vartical_index))

          if ((vartical_index % 216 == 0) and ((horizontal_index-15) % 270 == 0)) or vartical_index >= 540:

              fr = frame[vartical_index:vartical_index+215, horizontal_index:horizontal_index+269]
              cv2.imwrite(root_dir  + "/demovideo/temp_frame.jpg", fr)

              img = image.load_img(root_dir  + "/demovideo/temp_frame.jpg", target_size=(60, 70, 3))
              x = image.img_to_array(img)
              x = np.expand_dims(x, axis=0)

              features = model.predict(x)
              #print(features)
              max_features = np.amax(features)

              if max_features == 0:
                  predict_result = np.append(predict_result, 0)
                  cv2.circle(frame, (horizontal_index+135, vartical_index+108), 5, (255,255,255), thickness=-1)
                  cv2.putText(frame, "error", (horizontal_index+135+5, vartical_index+108+5), font, font_size,(255,255,255), font_boldness)
              elif features[0,0] == max_features:
                  predict_result = np.append(predict_result, 1)
                  cv2.circle(frame, (horizontal_index+135, vartical_index+108), 5,(160,255,60), thickness=-1)
                  cv2.putText(frame, categories[0], (horizontal_index+135+5, vartical_index+108+5), font, font_size,(160,255,60), font_boldness)
              elif features[0,1] == max_features:
                  predict_result = np.append(predict_result, 2)
                  cv2.circle(frame, (horizontal_index+135, vartical_index+108), 5,(160,255,60), thickness=-1)
                  cv2.putText(frame, categories[1], (horizontal_index+135+5, vartical_index+108+5), font, font_size,(160,255,60), font_boldness)
              elif features[0,2] == max_features:
                  predict_result = np.append(predict_result, 3)
                  cv2.circle(frame, (horizontal_index+135, vartical_index+108), 5, (220,64,255), thickness=-1)
                  cv2.putText(frame, categories[2], (horizontal_index+135+5, vartical_index+108+5), font, font_size,(220,64,255), font_boldness)
              elif features[0,3] == max_features:
                  predict_result = np.append(predict_result, 4)
                  cv2.circle(frame, (horizontal_index+135, vartical_index+108), 5, (180,140,255), thickness=-1)
                  cv2.putText(frame, categories[3], (horizontal_index+135+5, vartical_index+108+5), font, font_size, (180,140,255), font_boldness)
              elif features[0,4] == max_features:
                  predict_result = np.append(predict_result, 5)
                  cv2.circle(frame, (horizontal_index+135, vartical_index+108), 5, (128,128,0), thickness=-1)
                  cv2.putText(frame, categories[4], (horizontal_index+135+5, vartical_index+108+5), font, font_size, (128,128,0), font_boldness)
              elif features[0,5] == max_features:
                  predict_result = np.append(predict_result, 6)
                  cv2.circle(frame, (horizontal_index+135, vartical_index+108), 5, (128,255,128), thickness=-1)
                  cv2.putText(frame, categories[5], (horizontal_index+135+5, vartical_index+108+5), font, font_size,(128,255,128), font_boldness)
              elif features[0,6] == max_features:
                  predict_result = np.append(predict_result, 7)
                  cv2.circle(frame, (horizontal_index+135, vartical_index+108), 5,(0,0,0), thickness=-1)
                  cv2.putText(frame, categories[6], (horizontal_index+135+3, vartical_index+108+5), font, font_size,(0,0,0), font_boldness)
              elif features[0,7] == max_features:
                  predict_result = np.append(predict_result, 8)
                  cv2.circle(frame, (horizontal_index+135, vartical_index+108), 5, (128,255,128), thickness=-1)
                  cv2.putText(frame, categories[7], (horizontal_index+135+5, vartical_index+108+5), font, font_size,(128,255,128), font_boldness)
              else:
                  predict_result = np.append(predict_result, 0)
                  cv2.circle(frame, (horizontal_index+135, vartical_index+108), 5, (255,255,255), thickness=-1)
                  cv2.putText(frame, "error", (horizontal_index+135+5, vartical_index+108+5), font, font_size,(255,255,255), font_boldness)

          else:
              predict_result = np.append(predict_result, 0)
              #print("out")

    predict_result = predict_result.reshape(9,13)
    #print(predict_result)

    #Write for movie
    mov_out.write(frame)

    #if Esc Key is pushed, Escape loop
    key = cv2.waitKey(INTERVAL)
    if key == ESC_KEY:
        break

    #Load next video-frame
    ret, frame = mov_org.read()

#End
cv2.destroyAllWindows()
mov_org.release()
mov_out.release()

Using TensorFlow backend.







Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.








In [0]:
import tensorflow as tf
from tensorflow import keras
from keras import models
from keras.models import model_from_json
from keras.preprocessing import image
import numpy as np

root_dir="./gdrive/My Drive/Google Colab/CO191219_proj"
#Load saved-model
model = model_from_json(open(root_dir+'/Conv2D3-Dence3-Adadelta_v3/img_predict_Conv2D3-Dence3-Adadelta.json').read())
#Load saved-weight
model.load_weights(root_dir+'/Conv2D3-Dence3-Adadelta_v3/img_predict_Conv2D3-Dence3-Adadelta.hdf5')
#Define categories
categories = ["floor1", "floor2", "floor-object", "object", "wall", "hole", "human","ceiling"]

#Load image
img_path = root_dir + "/ImageforLearn_v2" + str(input())
img = image.load_img(img_path,target_size=(60, 70, 3))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)

#Predict
features = model.predict(x)
print(features)

max_features = np.amax(features)

#Output message
if max_features == 0:
    print("error")
elif features[0,0] == max_features:
    print (categories[0])
elif features[0,1] == max_features:
    print (categories[1])
elif features[0,2] == max_features:
    print (categories[2])
elif features[0,3] == max_features:
    print (categories[3])
elif features[0,4] == max_features:
    print (categories[4])
elif features[0,5] == max_features:
    print (categories[5])
elif features[0,6] == max_features:
    print (categories[6])
elif features[0,7] == max_features:
    print (categories[7])
else:
    print("error")

/floor1/_0_1.jpg
[[0. 0. 0. 1. 0. 0. 0. 0.]]
object
